In [ ]:
#default_exp networks

In [ ]:
import timm
import pretrainedmodels

In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import List

import torch
from torch import nn
import torch.nn.functional as F

from src.core import *
from src.layers import *

<IPython.core.display.Javascript object>

In [ ]:
# export
activation_map = dict(relu=nn.ReLU, mish=Mish, silu=nn.SiLU)

<IPython.core.display.Javascript object>

In [ ]:
activation_map

{'relu': torch.nn.modules.activation.ReLU,
 'mish': src.layers.Mish,
 'silu': torch.nn.modules.activation.SiLU}

<IPython.core.display.Javascript object>

In [ ]:
# export
class TransferLearningModel(nn.Module):
    "Transfer Learning with `encoder`"

    def __init__(
        self,
        encoder: nn.Module,
        c: int,
        cut: int = -2,
        act: nn.Module = nn.ReLU(inplace=True),
        lin_ftrs: int = 512,
    ):
        """
        Args:
            encoder (nn.Module) : the classifer to extract features.
            c (int) : number of output classes.
            cut (int) : number of layers to cut/keep from the encoder.
            act (nn.Module) : activation function for the head.
            lin_ftrs (int): linear features for the fc layer
        """
        super(TransferLearningModel, self).__init__()
        self.c = c
        self.encoder = encoder
        self.encoder = cut_model(self.encoder, cut)

        # create the custom head for the model
        feats = num_features_model(self.encoder, in_chs=3) * 2
        self.fc = create_head(feats, n_out=self.c, lin_ftrs=lin_ftrs, act=act)

        if isinstance(act, nn.ReLU):
            apply_init(self.fc, torch.nn.init.kaiming_normal_)
        else:
            apply_init(self.fc, torch.nn.init.kaiming_uniform_)

    def forward(self, xb):
        return self.fc(self.encoder(xb))

<IPython.core.display.Javascript object>

In [ ]:
dummpy_input = torch.zeros((2, 3, 255, 255))
base_model = timm.create_model("efficientnet_b0", drop_rate=0.25)

<IPython.core.display.Javascript object>

In [ ]:
model = TransferLearningModel(base_model, c=5, cut=-2)
model

TransferLearningModel(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
    (3): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          

<IPython.core.display.Javascript object>

In [ ]:
model(dummpy_input).data.shape

torch.Size([2, 5])

<IPython.core.display.Javascript object>

In [ ]:
base_model = timm.create_model(
    "tf_efficientnet_b0_ns",
    act_layer=activation_map["mish"],
    pretrained=True,
    drop_rate=0.25,
)
model = TransferLearningModel(base_model, c=5, cut=-2, act=Mish())
model

TransferLearningModel(
  (encoder): Sequential(
    (0): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): Mish()
    (3): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): Mish()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): Mish()
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (1): 

<IPython.core.display.Javascript object>

In [ ]:
# export
# TODO: add midlevel classification branch in learning.
class SnapMixTransferLearningModel(nn.Module):
    "Transfer Learning with model to be comaptible with Snapmix"

    def __init__(
        self, encoder: nn.Module, c: int, cut: int = -2, drop_rate=0.0, **kwargs
    ):
        """
        Args:
            encoder (nn.Module): the classifer to extract features
            c (int) : number of output classes
            cut (int) : number of layers to cut/keep from the encoder
        """
        super(SnapMixTransferLearningModel, self).__init__()
        self.c = c

        try:
            # for timm models
            try:
                feats = encoder.fc.in_features
            except:
                feats = encoder.classifier.in_features

        except:
            # for models from pretrainedmodels
            feats = encoder.last_linear.in_features

        # build model
        self.encoder = cut_model(encoder, cut)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flat_layer = nn.Flatten()

        if drop_rate > 0:
            self.dropout_layer = nn.Dropout(drop_rate)
        else:
            self.dropout_layer = None

        self.fc = nn.Linear(feats, self.c)
        apply_init(self.fc, torch.nn.init.kaiming_normal_)

    def forward(self, xb):
        fmps = self.encoder(xb)
        x = self.pool(fmps)
        x = self.flat_layer(x)
        if self.dropout_layer is not None:
            x = self.dropout_layer(x)
        return self.fc(x)

<IPython.core.display.Javascript object>

In [ ]:
base = timm.create_model("efficientnet_b0")
m = SnapMixTransferLearningModel(base, c=5, cut=-2)
m

SnapMixTransferLearningModel(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
    (3): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

<IPython.core.display.Javascript object>

In [ ]:
base = timm.create_model("efficientnet_b0", act_layer=Mish, drop_rate=0.25)
m2 = SnapMixTransferLearningModel(base, c=5, cut=-2, drop_rate=0.5)
m2

SnapMixTransferLearningModel(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Mish()
    (3): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): Mish()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): Mish()
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )


<IPython.core.display.Javascript object>

In [ ]:
m2(dummpy_input).shape, m(dummpy_input).shape

(torch.Size([2, 5]), torch.Size([2, 5]))

<IPython.core.display.Javascript object>

In [ ]:
# export
# from : https://www.kaggle.com/abhinand05/vision-transformer-vit-tutorial-baseline/output#Vision-Transformers:-A-gentle-introduction
class VisionTransformer(nn.Module):
    def __init__(self, encoder: nn.Module, c: int, **kwargs):
        super(VisionTransformer, self).__init__()
        self.model = encoder
        self.model.head = nn.Linear(self.model.head.in_features, c)
        apply_init(self.model.head, torch.nn.init.kaiming_normal_)

    def forward(self, xb):
        return self.model(xb)

<IPython.core.display.Javascript object>

In [ ]:
encoder = timm.create_model("vit_base_patch16_224", pretrained=True)
model = VisionTransformer(encoder, 5)

<IPython.core.display.Javascript object>

In [ ]:
model

VisionTransformer(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm(

<IPython.core.display.Javascript object>

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_layers.ipynb.
Converted 03a_networks.ipynb.
Converted 04_optimizers_schedules.ipynb.
Converted 05_lightning.data.ipynb.
Converted 05a_lightning.core.ipynb.
Converted 05b_lightning.callbacks.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>